In [1]:
import numpy as np

In [2]:
selected_file = "C:/2022_Summer_Intern/Gel_Images_UNet_Test/Models/PC_Gel_Nathan_Q1+Q2+selected_2022_07_28_16;42;07/segmentation_images/epoch1_mask_pred.npy"

In [ ]:
mask_pred = cv2.imread()  # H, W, C

In [3]:


data = np.load(selected_file)
print(data)

[[[-0.28026244  0.17395368]
  [-0.40589482  0.1103431 ]
  [-0.63686794  0.16847906]
  ...
  [-0.48301074  0.18597984]
  [-0.27459344  0.33381668]
  [-0.34943214  0.26269472]]

 [[-0.3485656   0.25797158]
  [-0.41043925  0.08262721]
  [-0.19145283  0.19203459]
  ...
  [-0.39125922  0.19966538]
  [-0.15522069  0.21579155]
  [-0.26822335  0.29285717]]

 [[-0.33475825  0.29778287]
  [-0.23148748  0.20576677]
  [-0.2161604   0.1291847 ]
  ...
  [-0.27662584  0.11311433]
  [-0.07216033  0.22480172]
  [-0.15674084  0.2649768 ]]

 ...

 [[-0.40663087  0.23502408]
  [-0.20326968  0.09328975]
  [-0.19138628  0.14164576]
  ...
  [-0.30378422  0.10782146]
  [-0.12555791  0.18133518]
  [-0.31390968  0.16435026]]

 [[-0.3216762   0.27381408]
  [-0.16330115  0.08872144]
  [-0.11936514  0.09321263]
  ...
  [-0.2537508  -0.03796969]
  [-0.15909976  0.03025222]
  [-0.22620349  0.20388788]]

 [[-0.33222744  0.14489895]
  [-0.08373796  0.09603738]
  [-0.20670052  0.12645948]
  ...
  [-0.30428487  0.064207